In [149]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import zipfile
import time
from PIL import Image
from google.colab import drive

import torchvision as tv
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [150]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
zip_file = '/content/drive/My Drive/Colab Notebooks/archive.zip'

z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

print(os.listdir())

['.config', 'drive', 'dataset', 'sample_data']


In [152]:
os.listdir('dataset')

['single_prediction', 'training_set', 'test_set']

In [153]:
#пишем кастомный датасет
class Dataset2class(torch.utils.data.Dataset):
  def __init__(self, path_dir1:str, path_dir2:str):
    super().__init__()

    self.path_dir1 = path_dir1 #путь к папке dogs
    self.path_dir2 = path_dir2

    self.dir1_list = sorted(os.listdir(path_dir1))  #содежимое папки dogs(картинки)
    self.dir2_list = sorted(os.listdir(path_dir2))

  def __len__(self):
    return len(self.dir1_list) + len(self.dir2_list)

  def __getitem__(self, idx):
    if idx < len(self.dir1_list):
      class_id = 0 #dogs
      img_path = os.path.join(self.path_dir1, self.dir1_list[idx]) # путь к картинке
    else:
      class_id = 1  #cats
      idx -= len(self.dir1_list)
      img_path = os.path.join(self.path_dir2, self.dir2_list[idx])

    img = cv2.imread(img_path, cv2.IMREAD_COLOR) # запись цветной картинки, зная к ней путь
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # перевод из BGR в RGB(из-за библиотеки сv2)
    img = img.astype(np.float32)
    img = img/255.0                       #нормализация
    img = cv2.resize(img, (64,64), interpolation=cv2.INTER_AREA)
    img = img.transpose((2, 0, 1))
    t_img = torch.from_numpy(img)
    t_class_id = torch.tensor(class_id)

    return {'img': t_img, 'label': t_class_id}


In [154]:
train_dogs_path = 'dataset/training_set/dogs'
train_cats_path = 'dataset/training_set/cats'
test_dogs_path = 'dataset/test_set/dogs'
test_cats_path = 'dataset/test_set/cats'
train_ds_catsdogs = Dataset2class(train_dogs_path, train_cats_path)
test_ds_catsdogs = Dataset2class(test_dogs_path, test_cats_path)

In [155]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, batch_size = batch_size,
    shuffle=True, drop_last= True, num_workers=0
)
test_loader = torch.utils.data.DataLoader(
    test_ds_catsdogs, batch_size = batch_size,
    shuffle=True, drop_last= False, num_workers=0
)

In [156]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [157]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.act = nn.LeakyReLU(negative_slope=0.2)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv0 = nn.Conv2d(3, 32, 5, stride=1, padding=0)
    self.conv1 = nn.Conv2d(32, 32, 5, stride=1, padding=0)
    self.conv2 = nn.Conv2d(32, 64, 5, stride=1, padding=0)
    #self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0)

    self.adaptivepool = nn.AdaptiveAvgPool2d((1,1))
    self.flatten = nn.Flatten()
    self.linear1 = nn.Linear(64, 32)
    self.linear2 = nn.Linear(32, 10)
    self.linear3 = nn.Linear(10, 2)
  def forward(self,x):

    out = self.conv0(x)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv1(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv2(out)
    out = self.act(out)
    out = self.maxpool(out)

    #out = self.conv3(out)
    #out = self.act(out)

    out = self.adaptivepool(out)
    out = self.flatten(out)

    out = self.linear1(out)
    out = self.act(out)
    out = self.linear2(out)
    out = self.act(out)
    out = self.linear3(out)

    return out


In [158]:
net = ConvNet()
net = net.to(device)

In [159]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

In [160]:
num_epochs = 10

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (sample) in enumerate(train_loader):
        # Прямой запуск
        img = sample['img'].to(device)
        label = sample['label'].to(device)
        outputs = net(img)
        loss = loss_fn(outputs, label)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = label.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == label).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))


Epoch [1/10], Step [100/500], Loss: 0.7027, Accuracy: 43.75%
Epoch [1/10], Step [200/500], Loss: 0.7210, Accuracy: 37.50%
Epoch [1/10], Step [300/500], Loss: 0.6873, Accuracy: 56.25%
Epoch [1/10], Step [400/500], Loss: 0.6743, Accuracy: 62.50%
Epoch [1/10], Step [500/500], Loss: 0.6649, Accuracy: 68.75%
Epoch [2/10], Step [100/500], Loss: 0.7008, Accuracy: 37.50%
Epoch [2/10], Step [200/500], Loss: 0.7239, Accuracy: 50.00%
Epoch [2/10], Step [300/500], Loss: 0.6629, Accuracy: 62.50%
Epoch [2/10], Step [400/500], Loss: 0.6920, Accuracy: 43.75%
Epoch [2/10], Step [500/500], Loss: 0.6950, Accuracy: 56.25%
Epoch [3/10], Step [100/500], Loss: 0.9265, Accuracy: 37.50%
Epoch [3/10], Step [200/500], Loss: 0.6900, Accuracy: 50.00%
Epoch [3/10], Step [300/500], Loss: 0.7314, Accuracy: 56.25%
Epoch [3/10], Step [400/500], Loss: 0.6808, Accuracy: 75.00%
Epoch [3/10], Step [500/500], Loss: 0.6025, Accuracy: 62.50%
Epoch [4/10], Step [100/500], Loss: 0.6374, Accuracy: 56.25%
Epoch [4/10], Step [200/

In [161]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for sample in test_loader:
        img = sample['img'].to(device)
        label = sample['label'].to(device)
        outputs = net(img)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))


Test Accuracy of the model on the 10000 test images: 71.35000000000001 %
